In [ ]:
#common code
from typing import List, Union, Any, Tuple
import math
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm


def generate_distorted_grid(
    n_points_x: int, n_points_y: int, spacing: float, noise_standard_deviation: float, pillar_defect_prob: float, pillar_region_defect_prob: float
) -> Tuple[np.ndarray, Any]:
    grid_points = []
    grid_points_defect = defaultdict(dict)
    region_defect = np.zeros([REGION_SIZE_PERC_OF_GRID, REGION_SIZE_PERC_OF_GRID], dtype = bool) 
    #region_defect = [[False for _ in range(REGION_SIZE_PERC_OF_GRID)] for _ in range(REGION_SIZE_PERC_OF_GRID)]
    for y, x in np.ndindex(region_defect.shape):
        region_defect[y, x] = True if np.random.rand() < (pillar_region_defect_prob/100) else False

    assert(n_points_x == n_points_y)
    region_size =  math.ceil(n_points_x / REGION_SIZE_PERC_OF_GRID)

    for i in range(n_points_x):
        for j in range(n_points_y):
            x_base = i * spacing
            y_base = j * spacing

            x_noise = np.random.normal(0, noise_standard_deviation)
            y_noise = np.random.normal(0, noise_standard_deviation)

            x = x_base + x_noise
            y = y_base + y_noise

            grid_points_defect[x][y] = True if np.random.rand() < (pillar_defect_prob / 100) else False
            grid_points_defect[x][y] = True if region_defect[math.floor(i/region_size), math.floor(j/region_size)] else False
            grid_points.append([x, y])

    return np.array(grid_points), grid_points_defect


def point_in_rectangle(
    point: np.ndarray, rect_corners: np.ndarray, pillar_diameter: float = 0.0
) -> bool:
    x, y = point
    radius = pillar_diameter / 2.0

    def cross_product_sign(p1, p2, p3):
        return (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0])

    def point_to_segment_distance(p, seg_start, seg_end):
        px, py = p
        x1, y1 = seg_start
        x2, y2 = seg_end

        dx = x2 - x1
        dy = y2 - y1
        seg_length_sq = dx * dx + dy * dy

        if seg_length_sq == 0:
            return np.sqrt((px - x1) ** 2 + (py - y1) ** 2)

        t = max(0, min(1, ((px - x1) * dx + (py - y1) * dy) / seg_length_sq))

        proj_x = x1 + t * dx
        proj_y = y1 + t * dy

        return np.sqrt((px - proj_x) ** 2 + (py - proj_y) ** 2)

    sign1 = cross_product_sign(rect_corners[0], rect_corners[1], point)
    sign2 = cross_product_sign(rect_corners[1], rect_corners[2], point)
    sign3 = cross_product_sign(rect_corners[2], rect_corners[3], point)
    sign4 = cross_product_sign(rect_corners[3], rect_corners[0], point)

    has_neg = (sign1 < 0) or (sign2 < 0) or (sign3 < 0) or (sign4 < 0)
    has_pos = (sign1 > 0) or (sign2 > 0) or (sign3 > 0) or (sign4 > 0)

    point_inside_rect = not (has_neg and has_pos)

    if radius == 0:
        return point_inside_rect

    if point_inside_rect:
        return True

    edges = [
        (rect_corners[0], rect_corners[1]),
        (rect_corners[1], rect_corners[2]),
        (rect_corners[2], rect_corners[3]),
        (rect_corners[3], rect_corners[0]),
    ]

    for seg_start, seg_end in edges:
        dist = point_to_segment_distance(point, seg_start, seg_end)
        if dist <= radius:
            return True

    return False


def find_points_in_rectangle(
    grid_points: np.ndarray, grid_points_defect: Any, rect_corners: np.ndarray, pillar_diameter: float = 0.0
) -> List[np.ndarray]:
    x_min_rect = rect_corners[:, 0].min()
    x_max_rect = rect_corners[:, 0].max()
    y_min_rect = rect_corners[:, 1].min()
    y_max_rect = rect_corners[:, 1].max()

    x_coords = grid_points[:, 0]
    y_coords = grid_points[:, 1]

    candidate_mask = (
        (x_coords - pillar_diameter >= x_min_rect)
        & (x_coords + pillar_diameter <= x_max_rect)
        & (y_coords - pillar_diameter >= y_min_rect)
        & (y_coords + pillar_diameter <= y_max_rect)
    )

    candidate_points = grid_points[candidate_mask]

    points_inside = []
    for point in candidate_points:
        if point_in_rectangle(point, rect_corners, pillar_diameter) and grid_points_defect[point[0]][point[1]] is False: 
            points_inside.append(point)

    return points_inside


def generate_rectangle_corners(
    center_x: float, center_y: float, width: float, length: float, angle: float
) -> np.ndarray:
    half_width = width / 2
    half_length = length / 2

    local_corners = np.array(
        [
            [-half_width, -half_length],
            [half_width, -half_length],
            [half_width, half_length],
            [-half_width, half_length],
        ]
    )

    rotation_matrix = np.array(
        [[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]]
    )

    rotated_corners = local_corners @ rotation_matrix.T
    rect_corners = rotated_corners + np.array([center_x, center_y])

    return rect_corners


# pyre-ignore
def assess_bacterium_death(
    points_inside: List[np.ndarray], rect_corners: np.ndarray
) -> bool:
    if len(points_inside) < 2:
        return False

    points_array = np.array(points_inside)

    length_vector = rect_corners[2] - rect_corners[1]
    length_direction = length_vector / np.linalg.norm(length_vector)

    projections = points_array @ length_direction

    sorted_projections = np.sort(projections)

    distances = np.diff(sorted_projections)

    if len(distances) == 0:
        return False

    longest_distance = np.max(distances)
    stress = Two_Weight_Unit_Over_Pi * pow(longest_distance, 4) / ((BACTERIUM_DI * BACTERIUM_DI + BACTERIUM_DO * BACTERIUM_DO) * (BACTERIUM_DI + BACTERIUM_DO))
    all_stress_samples.append(stress)
    return True if stress >= CRITICAL_STRESS else False


def run_iterations(
    grid_points: np.ndarray,
    grid_points_defect: any,
    rect_width: float,
    rect_length: float,
    n_bacterium: int,
    pillar_diameter: float,
) -> float:
    x_min = grid_points[:, 0].min()
    x_max = grid_points[:, 0].max()
    y_min = grid_points[:, 1].min()
    y_max = grid_points[:, 1].max()

    n_killed = 0

    for iteration in range(n_bacterium):
        center_x = np.random.uniform(x_min + rect_width, x_max - rect_width)
        center_y = np.random.uniform(y_min + rect_length, y_max - rect_length)

        angle = np.random.uniform(0, 2 * np.pi)

        rect_corners = generate_rectangle_corners(
            center_x, center_y, rect_width, rect_length, angle
        )

        points_inside = find_points_in_rectangle(
            grid_points, grid_points_defect, rect_corners, pillar_diameter
        )

        is_killed = assess_bacterium_death(points_inside, rect_corners)
        if is_killed:
            n_killed += 1

    kill_percentage = (n_killed / n_bacterium) * 100.0
    return kill_percentage


def ensure_list(value: Union[int, float, List]) -> List:
    if isinstance(value, list):
        return value
    return [value]


def plot_kill_efficiency(xlabel, ylabel, values, saved_filename):
    plt.figure(facecolor='white')
    
    plt.rcParams.update(
        {
            "font.family": "sans-serif",
            "font.sans-serif": ["Arial"],
            "font.size": 30,
            "axes.linewidth": 1.5,
            "xtick.major.width": 1.5,
            "ytick.major.width": 1.5,
            "xtick.major.size": 5,
            "ytick.major.size": 5,
        }
    )
    
    plt.plot(
        values,
        kill_percentages,
        linewidth=2.5,
        color="#2E86AB",
        marker="o",
        markersize=6,
        markerfacecolor="white",
        markeredgewidth=2,
        markeredgecolor="#2E86AB",
    )
    
    plt.xlabel(xlabel, fontsize=20, fontweight="bold")
    plt.ylabel(ylabel, fontsize=20, fontweight="bold")
    
    plt.xlim(0, max(values) * 1.05)
    plt.ylim(0, 100)
    
    plt.grid(True, alpha=0.3, linestyle="--", linewidth=0.8)
    ax = plt.gca()
    
    plt.tight_layout()
    plt.savefig(saved_filename, bbox_inches="tight", pad_inches=0)
    plt.show()

def run_simulation(
    grid_size: Union[int, List[int]],
    spacing: Union[float, List[float]],
    noise_standard_deviation: Union[float, List[float]],
    pillar_defect_prob: Union[float, List[float]],
    pillar_region_defect_prob: Union[float, List[float]],
    rect_width: Union[float, List[float]],
    rect_length: Union[float, List[float]],
    n_bacterium: int,
    pillar_diameter: Union[float, List[float]],
) -> List[dict]:
    grid_size_list = ensure_list(grid_size)
    spacing_list = ensure_list(spacing)
    noise_standard_deviation_list = ensure_list(noise_standard_deviation)
    pillar_defect_prob_list = ensure_list(pillar_defect_prob)
    rect_width_list = ensure_list(rect_width)
    rect_length_list = ensure_list(rect_length)
    pillar_diameter_list = ensure_list(pillar_diameter)

    results = []

    for gs in grid_size_list:
        for sp in spacing_list:
            for nsd in noise_standard_deviation_list:
                for pdp in pillar_defect_prob_list:
                    for prdp in pillar_region_defect_prob:
                        print("=" * 80)
                        print(
                            f"Configuration: GRID_SIZE={gs}, SPACING={sp}, NOISE_STANDARD_DEVIATION={nsd}, PILLAR_DEFECT_PROB={pdp}, PILLAR_REGION_DEFECT_PROB={prdp}"
                        )
                        print("=" * 80)
        
                        grid_points, grid_points_defect = generate_distorted_grid(gs, gs, sp, nsd, pdp, prdp)
        
                        x_min = grid_points[:, 0].min()
                        x_max = grid_points[:, 0].max()
                        y_min = grid_points[:, 1].min()
                        y_max = grid_points[:, 1].max()
        
                        for rw in rect_width_list:
                            for rl in rect_length_list:
                                for pd in pillar_diameter_list:
                                    kill_percentage = run_iterations(
                                        grid_points, grid_points_defect, rw, rl, n_bacterium, pd
                                    )
        
                                    print(f"\nKill percentage: {kill_percentage:.2f}%\n")
        
                                    result = {
                                        "grid_size": gs,
                                        "spacing": sp,
                                        "noise_standard_deviation": nsd,
                                        "pillar_defect_prob": pdp,
                                         "pillar_region_defect_prob": prdp,
                                        "rect_width": rw,
                                        "rect_length": rl,
                                        "pillar_diameter": pd,
                                        "kill_percentage": kill_percentage,
                                    }
                                    results.append(result)

    return results


GRID_SIZE = 500
SPACING = 10.0
RECT_WIDTH = 500
RECT_LENGTH = 3000
N_BACTERIUM = 100
PILLAR_DIAMETER = 36
BACTERIUM_DO = 500
BACTERIUM_DI = 490
WEIGHT_UNIT = 1.0
CRITICAL_STRESS =  0.094 # for space ~1000

#Variables simulating nano-surface defects
NOISE_STANDARD_DEVIATION = 0.0 # pillar positions subject to a Gaussian noise
PILLAR_DEFECT_PROB = 0 # Prob. for an existing pillar failing to be created 
PILLAR_REGION_DEFECT_PROB = 0 # Prob. for a region of pillars failing to be created (always divide the grid into 10*10 regions) 
REGION_SIZE_PERC_OF_GRID = 10  # 10 means evenly dividing the grid into 10*10 regions 


Two_Weight_Unit_Over_Pi = 2 * WEIGHT_UNIT / math.pi # constant part in the stress computation. Used for reducing computing overhead

all_stress_samples = []

In [ ]:
# Vary PILLAR_DEFECT_PROB

SPACING = 92
PILLAR_DIAMETER = 36
BACTERIUM_DO = 500
BACTERIUM_DI = 490

NOISE_STANDARD_DEVIATION = 0.0 # pillar positions subject to a Gaussian noise
PILLAR_DEFECT_PROB = 0 # In [0, 100]. Prob. for an existing pillar failing to be created 

np_array = np.linspace(5, 2000, num=30)
SPACING = np_array.tolist() 

results = run_simulation(
    grid_size=GRID_SIZE,
    spacing=SPACING,
    noise_standard_deviation=NOISE_STANDARD_DEVIATION,
    pillar_defect_prob=PILLAR_DEFECT_PROB,
    pillar_region_defect_prob=PILLAR_REGION_DEFECT_PROB,
    rect_width=RECT_WIDTH,
    rect_length=RECT_LENGTH,
    n_bacterium=N_BACTERIUM,
    pillar_diameter=PILLAR_DIAMETER,
)

print("\n" + "=" * 80)
print("SUMMARY OF RESULTS")
print("=" * 80)
for result in results:
    print(f"\nConfiguration:")
    print(f"  Grid Size: {result['grid_size']}")
    print(f"  Spacing: {result['spacing']}")
    print(f"  Noise Standard Deviation: {result['noise_standard_deviation']}")
    print(f"  Pillar Defect Prob: {result['pillar_defect_prob']}")
    print(f"  Pillar Region Defect Prob: {result['pillar_region_defect_prob']}")
    print(f"  Rect Width: {result['rect_width']}")
    print(f"  Rect Length: {result['rect_length']}")
    print(f"  Pillar Diameter: {result['pillar_diameter']}")
    print(f"  Kill Percentage: {result['kill_percentage']:.2f}%")

values = [result["spacing"] for result in results]
kill_percentages = [result["kill_percentage"] for result in results]

In [ ]:
# Plot killing percentage vs pillar diameter
xlabel = "Pillar-to-pillar spacing (nm)"
ylabel = "Killing percentage (%)"
saved_filename = "killing_perc_vs_p2p_spacing.png"
plot_kill_efficiency(xlabel, ylabel, values, saved_filename)

In [ ]:
# Vary pillar diameter

GRID_SIZE = 500
SPACING = 1000 
NOISE_STANDARD_DEVIATION = 0.0
RECT_WIDTH = 500
RECT_LENGTH = 3000
N_BACTERIUM = 1000
PILLAR_DIAMETER = 36
BACTERIUM_DO = 500
BACTERIUM_DI = 490

NOISE_STANDARD_DEVIATION = 0.0 # pillar positions subject to a Gaussian noise
PILLAR_DEFECT_PROB = 0 # In [0, 100]. Prob. for an existing pillar failing to be created 

np_array = np.linspace(0, 200, num=20)
PILLAR_DIAMETER = np_array.tolist() 

results = run_simulation(
    grid_size=GRID_SIZE,
    spacing=SPACING,
    noise_standard_deviation=NOISE_STANDARD_DEVIATION,
    pillar_defect_prob=PILLAR_DEFECT_PROB,
    pillar_region_defect_prob=PILLAR_REGION_DEFECT_PROB,
    rect_width=RECT_WIDTH,
    rect_length=RECT_LENGTH,
    n_bacterium=N_BACTERIUM,
    pillar_diameter=PILLAR_DIAMETER,
)

print("\n" + "=" * 80)
print("SUMMARY OF RESULTS")
print("=" * 80)
for result in results:
    print(f"\nConfiguration:")
    print(f"  Grid Size: {result['grid_size']}")
    print(f"  Spacing: {result['spacing']}")
    print(f"  Noise Standard Deviation: {result['noise_standard_deviation']}")
    print(f"  Pillar Defect Prob: {result['pillar_defect_prob']}")
    print(f"  Pillar Region Defect Prob: {result['pillar_region_defect_prob']}")
    print(f"  Rect Width: {result['rect_width']}")
    print(f"  Rect Length: {result['rect_length']}")
    print(f"  Pillar Diameter: {result['pillar_diameter']}")
    print(f"  Kill Percentage: {result['kill_percentage']:.2f}%")

values = [result["pillar_diameter"] for result in results]
kill_percentages = [result["kill_percentage"] for result in results]

In [ ]:
# Plot killing percentage vs pillar diameter
xlabel = "Pillar diameter (nm)"
ylabel = "Killing percentage (%)"
saved_filename = "killing_perc_vs_pillar_diameter.png"
plot_kill_efficiency(xlabel, ylabel, values, saved_filename)

In [ ]:
# Vary noise_standard_deviation applied to pillar location

GRID_SIZE = 500
SPACING = 1000
RECT_WIDTH = 500
RECT_LENGTH = 3000
N_BACTERIUM = 1000
PILLAR_DIAMETER = 36
NOISE_STANDARD_DEVIATION = np.linspace(0, SPACING * 5, num=20).tolist()
PILLAR_DEFECT_PROB = 0 # In [0, 100]. Prob. for an existing pillar failing to be created 
BACTERIUM_DO = 500
BACTERIUM_DI = 490

results = run_simulation(
    grid_size=GRID_SIZE,
    spacing=SPACING,
    noise_standard_deviation=NOISE_STANDARD_DEVIATION,
    pillar_defect_prob=PILLAR_DEFECT_PROB,
    pillar_region_defect_prob=PILLAR_REGION_DEFECT_PROB,
    rect_width=RECT_WIDTH,
    rect_length=RECT_LENGTH,
    n_bacterium=N_BACTERIUM,
    pillar_diameter=PILLAR_DIAMETER,
)

print("\n" + "=" * 80)
print("SUMMARY OF RESULTS")
print("=" * 80)
for result in results:
    print(f"\nConfiguration:")
    print(f"  Grid Size: {result['grid_size']}")
    print(f"  Spacing: {result['spacing']}")
    print(f"  Noise Standard Deviation: {result['noise_standard_deviation']}")
    print(f"  Pillar Defect Prob: {result['pillar_defect_prob']}")
    print(f"  Pillar Region Defect Prob: {result['pillar_region_defect_prob']}")
    print(f"  Rect Width: {result['rect_width']}")
    print(f"  Rect Length: {result['rect_length']}")
    print(f"  Pillar Diameter: {result['pillar_diameter']}")
    print(f"  Kill Percentage: {result['kill_percentage']:.2f}%")

values = [result["noise_standard_deviation"] for result in results]
kill_percentages = [result["kill_percentage"] for result in results]

In [ ]:
# Plot killing percentage vs pillar diameter
xlabel = "Standard deviation of pillar position (nm)"
ylabel = "Killing percentage (%)"
saved_filename = "killing_perc_vs_pillar_position_std.png"
plot_kill_efficiency(xlabel, ylabel, values, saved_filename)

In [ ]:
# Vary defect/um^2 via PILLAR_DEFECT_PROB

GRID_SIZE = 500
SPACING = 1000 
RECT_WIDTH = 500
RECT_LENGTH = 3000
N_BACTERIUM = 1000
PILLAR_DIAMETER = 36
BACTERIUM_DO = 500
BACTERIUM_DI = 490

NOISE_STANDARD_DEVIATION = 0.0 # pillar positions subject to a Gaussian noise
PILLAR_DEFECT_PROB = 0 # In [0, 100]. Prob. for an existing pillar failing to be created 

np_array = np.linspace(0, 30, num=20)
PILLAR_DEFECT_PROB = np_array.tolist() 


all_stress_samples.clear()

results = run_simulation(
    grid_size=GRID_SIZE,
    spacing=SPACING,
    noise_standard_deviation=NOISE_STANDARD_DEVIATION,
    pillar_defect_prob=PILLAR_DEFECT_PROB,
    pillar_region_defect_prob=PILLAR_REGION_DEFECT_PROB,
    rect_width=RECT_WIDTH,
    rect_length=RECT_LENGTH,
    n_bacterium=N_BACTERIUM,
    pillar_diameter=PILLAR_DIAMETER,
)

print("\n" + "=" * 80)
print("SUMMARY OF RESULTS")
print("=" * 80)
for result in results:
    print(f"\nConfiguration:")
    print(f"  Grid Size: {result['grid_size']}")
    print(f"  Spacing: {result['spacing']}")
    print(f"  Noise Standard Deviation: {result['noise_standard_deviation']}")
    print(f"  Pillar Defect Prob: {result['pillar_defect_prob']}")
    print(f"  Pillar Region Defect Prob: {result['pillar_region_defect_prob']}")
    print(f"  Rect Width: {result['rect_width']}")
    print(f"  Rect Length: {result['rect_length']}")
    print(f"  Pillar Diameter: {result['pillar_diameter']}")
    print(f"  Kill Percentage: {result['kill_percentage']:.2f}%")

print("50% stress samples:", np.percentile(np.array(all_stress_samples), 50))

values = [result["pillar_defect_prob"] * SPACING * SPACING * 0.000001 for result in results]
kill_percentages = [result["kill_percentage"] for result in results]

In [ ]:
# Plot killing percentage vs pillar defect prob
xlabel = "Nano-surface defect density (defects/um^2)"
ylabel = "Killing percentage (%)"
saved_filename = "killing_perc_vs_pillar_defect_density.png"
plot_kill_efficiency(xlabel, ylabel, values, saved_filename)

In [ ]:
# Vary both noise_standard_deviation and pillar defect applied to pillar location

GRID_SIZE = 500
SPACING = 1000
RECT_WIDTH = 500
RECT_LENGTH = 3000
N_BACTERIUM = 3000
PILLAR_DIAMETER = 36
BACTERIUM_DO = 500
BACTERIUM_DI = 490

NOISE_STANDARD_DEVIATION = np.linspace(0, SPACING * 5, num=20).tolist()
PILLAR_DEFECT_PROB = np.linspace(0, 30, num=20).tolist()

results = run_simulation(
    grid_size=GRID_SIZE,
    spacing=SPACING,
    noise_standard_deviation=NOISE_STANDARD_DEVIATION,
    pillar_defect_prob=PILLAR_DEFECT_PROB,
    pillar_region_defect_prob=PILLAR_REGION_DEFECT_PROB,
    rect_width=RECT_WIDTH,
    rect_length=RECT_LENGTH,
    n_bacterium=N_BACTERIUM,
    pillar_diameter=PILLAR_DIAMETER,
)

print("\n" + "=" * 80)
print("SUMMARY OF RESULTS")
print("=" * 80)
for result in results:
    print(f"\nConfiguration:")
    print(f"  Grid Size: {result['grid_size']}")
    print(f"  Spacing: {result['spacing']}")
    print(f"  Noise Standard Deviation: {result['noise_standard_deviation']}")
    print(f"  Pillar Defect Prob: {result['pillar_defect_prob']}")
    print(f"  Pillar Region Defect Prob: {result['pillar_region_defect_prob']}")
    print(f"  Rect Width: {result['rect_width']}")
    print(f"  Rect Length: {result['rect_length']}")
    print(f"  Pillar Diameter: {result['pillar_diameter']}")
    print(f"  Kill Percentage: {result['kill_percentage']:.2f}%")



In [ ]:
# Plot 3D map when both noise_standard_deviation and pillar defect varies
from matplotlib import cm
x = [result["noise_standard_deviation"] for result in results]
#y = [result["pillar_defect_prob"] for result in results]
y = [result["pillar_defect_prob"] * SPACING * SPACING * 0.000001 for result in results] # displayed y value
z = [result["kill_percentage"] for result in results]
x_2d = np.array(x).reshape(20, 20)
y_2d = np.array(y).reshape(20, 20)
z_2d = np.array(z).reshape(20, 20)

xlabel = "Pillar position STD (nm)"
ylabel = "Nano-surface defect density (per um^2)"
zlabel = "Killing percentage (%)"
saved_filename = "3d_killing_perc_vs_co_vairation_std_defectDensity.png"

#X, Y = np.meshgrid(x, y)
#Z = f(X, Y)
fig = plt.figure()
ax = plt.axes(projection='3d')
#ax.contour3D(x_2d, y_2d, z_2d, 50, cmap='binary')
#ax.plot_wireframe(x_2d, y_2d, z_2d)
#ax.plot_surface(x_2d, y_2d, z_2d, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
surf = ax.plot_surface(x_2d, y_2d, z_2d, cmap=cm.coolwarm, linewidth=0, antialiased=False)
ax.set_xlabel(xlabel, labelpad=15)
ax.set_ylabel(ylabel, labelpad=15)
ax.set_zlabel(zlabel)

#plt.rcParams.update(
#    {
#        "font.family": "sans-serif",
#        "font.sans-serif": ["Arial"],
#        "font.size": 20,
#        "axes.linewidth": 1.5,
#        "xtick.major.width": 1.5,
#        "ytick.major.width": 1.5,
#        "xtick.major.size": 5,
#        "ytick.major.size": 5,
#    }
#)

plt.xlim(0, max(x) * 1.05)
plt.ylim(0, max(y) * 1.05)
ax.view_init(elev=10., azim=20)

plt.tight_layout()
plt.savefig(saved_filename, bbox_inches="tight", pad_inches=0)
#plt.savefig(saved_filename)
plt.show()



In [ ]:
# Vary RECT_LENGTH (Bacterium length)

GRID_SIZE = 500
SPACING = 1000 
RECT_WIDTH = 500
RECT_LENGTH = 3000
N_BACTERIUM = 1000
PILLAR_DIAMETER = 36
BACTERIUM_DO = 500
BACTERIUM_DI = 490

NOISE_STANDARD_DEVIATION = 0.0 # pillar positions subject to a Gaussian noise
PILLAR_DEFECT_PROB = 0 # In [0, 100]. Prob. for an existing pillar failing to be created 

np_array = np.linspace(500, 10000, num=20)
RECT_LENGTH = np_array.tolist() 


all_stress_samples.clear()

results = run_simulation(
    grid_size=GRID_SIZE,
    spacing=SPACING,
    noise_standard_deviation=NOISE_STANDARD_DEVIATION,
    pillar_defect_prob=PILLAR_DEFECT_PROB,
    pillar_region_defect_prob=PILLAR_REGION_DEFECT_PROB,
    rect_width=RECT_WIDTH,
    rect_length=RECT_LENGTH,
    n_bacterium=N_BACTERIUM,
    pillar_diameter=PILLAR_DIAMETER,
)

print("\n" + "=" * 80)
print("SUMMARY OF RESULTS")
print("=" * 80)
for result in results:
    print(f"\nConfiguration:")
    print(f"  Grid Size: {result['grid_size']}")
    print(f"  Spacing: {result['spacing']}")
    print(f"  Noise Standard Deviation: {result['noise_standard_deviation']}")
    print(f"  Pillar Defect Prob: {result['pillar_defect_prob']}")
    print(f"  Pillar Region Defect Prob: {result['pillar_region_defect_prob']}")
    print(f"  Rect Width: {result['rect_width']}")
    print(f"  Rect Length: {result['rect_length']}")
    print(f"  Pillar Diameter: {result['pillar_diameter']}")
    print(f"  Kill Percentage: {result['kill_percentage']:.2f}%")


values = [result["rect_length"] for result in results]
kill_percentages = [result["kill_percentage"] for result in results]

In [ ]:
# Plot killing percentage vs bacterium length
xlabel = "Bacterium length (nm)"
ylabel = "Killing percentage (%)"
saved_filename = "killing_perc_vs_bacterium_length.png"
plot_kill_efficiency(xlabel, ylabel, values, saved_filename)

In [ ]:
# Vary RECT_WIDTH (Bacterium outside diameter)

GRID_SIZE = 500
SPACING = 1000 
RECT_WIDTH = 500
RECT_LENGTH = 3000
N_BACTERIUM = 1000
PILLAR_DIAMETER = 36
BACTERIUM_DO = 500
BACTERIUM_DI = 490

NOISE_STANDARD_DEVIATION = 0.0 # pillar positions subject to a Gaussian noise
PILLAR_DEFECT_PROB = 0 # In [0, 100]. Prob. for an existing pillar failing to be created 

np_array = np.linspace(50, 1000, num=20)
RECT_WIDTH = np_array.tolist() 


all_stress_samples.clear()

results = run_simulation(
    grid_size=GRID_SIZE,
    spacing=SPACING,
    noise_standard_deviation=NOISE_STANDARD_DEVIATION,
    pillar_defect_prob=PILLAR_DEFECT_PROB,
    pillar_region_defect_prob=PILLAR_REGION_DEFECT_PROB,
    rect_width=RECT_WIDTH,
    rect_length=RECT_LENGTH,
    n_bacterium=N_BACTERIUM,
    pillar_diameter=PILLAR_DIAMETER,
)

print("\n" + "=" * 80)
print("SUMMARY OF RESULTS")
print("=" * 80)
for result in results:
    print(f"\nConfiguration:")
    print(f"  Grid Size: {result['grid_size']}")
    print(f"  Spacing: {result['spacing']}")
    print(f"  Noise Standard Deviation: {result['noise_standard_deviation']}")
    print(f"  Pillar Defect Prob: {result['pillar_defect_prob']}")
    print(f"  Pillar Region Defect Prob: {result['pillar_region_defect_prob']}")
    print(f"  Rect Width: {result['rect_width']}")
    print(f"  Rect Length: {result['rect_length']}")
    print(f"  Pillar Diameter: {result['pillar_diameter']}")
    print(f"  Kill Percentage: {result['kill_percentage']:.2f}%")


values = [result["rect_width"] for result in results]
kill_percentages = [result["kill_percentage"] for result in results]

In [ ]:
# Plot killing percentage vs bacterium length
xlabel = "Bacterium outside diameter (nm)"
ylabel = "Killing percentage (%)"
saved_filename = "killing_perc_vs_bacterium_outside_diameter.png"
plot_kill_efficiency(xlabel, ylabel, values, saved_filename)

In [ ]:
# Vary 

GRID_SIZE = 500
SPACING = 1000 
RECT_WIDTH = 500
RECT_LENGTH = 3000
N_BACTERIUM = 1000
PILLAR_DIAMETER = 36
BACTERIUM_DO = 500
BACTERIUM_DI = 490

NOISE_STANDARD_DEVIATION = 0.0 # pillar positions subject to a Gaussian noise
PILLAR_DEFECT_PROB = 0 # In [0, 100]. Prob. for an existing pillar failing to be created 

PILLAR_REGION_DEFECT_PROB = np.linspace(0, 100, num=20)

all_stress_samples.clear()

results = run_simulation(
    grid_size=GRID_SIZE,
    spacing=SPACING,
    noise_standard_deviation=NOISE_STANDARD_DEVIATION,
    pillar_defect_prob=PILLAR_DEFECT_PROB,
    pillar_region_defect_prob=PILLAR_REGION_DEFECT_PROB,
    rect_width=RECT_WIDTH,
    rect_length=RECT_LENGTH,
    n_bacterium=N_BACTERIUM,
    pillar_diameter=PILLAR_DIAMETER,
)

print("\n" + "=" * 80)
print("SUMMARY OF RESULTS")
print("=" * 80)
for result in results:
    print(f"\nConfiguration:")
    print(f"  Grid Size: {result['grid_size']}")
    print(f"  Spacing: {result['spacing']}")
    print(f"  Noise Standard Deviation: {result['noise_standard_deviation']}")
    print(f"  Pillar Defect Prob: {result['pillar_defect_prob']}")
    print(f"  Pillar Region Defect Prob: {result['pillar_region_defect_prob']}")
    print(f"  Rect Width: {result['rect_width']}")
    print(f"  Rect Length: {result['rect_length']}")
    print(f"  Pillar Diameter: {result['pillar_diameter']}")
    print(f"  Kill Percentage: {result['kill_percentage']:.2f}%")


values = [result["pillar_region_defect_prob"] for result in results]
kill_percentages = [result["kill_percentage"] for result in results]

In [ ]:
# Plot killing percentage vs bacterium length
xlabel = "Nano-surface regional defect percentage (%)"
ylabel = "Killing percentage (%)"
saved_filename = "killing_perc_vs_pillar_region_defect_prob.png"
plot_kill_efficiency(xlabel, ylabel, values, saved_filename)